In [ ]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import io

In [ ]:
class LSTM():
    def __init__(self):
        self.batch_size = 0
        self.window_size = 0
        self.hidden_layers = 0
        self.clip_margin = 0
        self.learning_rate = 0
        self.epochs = 0
    ############################################################################
    # Defining hyperparameters, THEY SHOULD BE MODIFIED
    batch_size = 10  # number of windows passed at once
    window_size = 10  # number of data points we want to predict
    hidden_layers = 250  # number of hidden layers in the LSTM
    clip_margin = 4  # used to clip gradients above/below its margins
    learning_rate = 0.0001  # way to optimize the loss function
    epochs = 200  # number of iterations the model needs to make (forward and back propagation)

    ############################################################################


In [ ]:
################## Windowing function ####################################
    def windowed_data(data, window_size):
        X = []
        y = []
        i = 0

        while (i + window_size) <= len(data) - 1:
            X.append(data[i:i + window_size])
            y.append(data[i + window_size])

            i += 1
        assert len(y) == len(X)
        return X, y

    ##########################################################################

In [ ]:
################## Sigmoid function ######################################
    def calcSig(inputs, outputs, wGate, wHidden, bias):
        gate = tf.sigmoid(tf.matmul(wGate, inputs) + tf.matmul(wHidden, outputs) + bias)
        return gate


In [ ]:
################## LSTM-block function ###################################
    def block(inputs, outputs, cell_state, array_of_weights, array_of_bias): # hidden_layers
        inputG = calcSig(inputs, outputs, array_of_weights[0], array_of_weights[1], array_of_bias[0])
        forgetG = calcSig(inputs, outputs, array_of_weights[2], array_of_weights[3], array_of_bias[1])
        outputG = calcSig(inputs, outputs, array_of_weights[4], array_of_weights[5], array_of_bias[2])
        memoryG = calcSig(inputs, outputs, array_of_weights[6], array_of_weights[7], array_of_bias[3])

        cell_state = cell_state * forgetG + inputG * memoryG    # final cell state C
        output_f = tf.tanh(cell_state) * outputG                # final output h

        return cell_state, output_f

    ##########################################################################

In [ ]:
 # Loading data and converting it into a .csv file
    data_xls = pd.read_excel('Sarcos.xls', 'Sheet1', index_col=None)
    # data_xls.to_csv('Sarcos_csv.csv', encoding='utf-8')

    data1 = data_xls.iloc[:, 1]  # reading from index 1 in the csv file
    # print(data1)

    scaler = StandardScaler()
    scaled_data1 = scaler.fit_transform(data1.values.reshape(-1, 1))

    ############################################################################

In [ ]:
# Splitting into training and validation data

    X, y = test.windowed_data(data1, window_size)
    size_of_80p = int(len(scaled_data1) * 0.8)

    X_train = np.array(X[:size_of_80p])  # we want 80 % to be training data
    y_train = np.array(y[:size_of_80p])

    X_test = np.array(X[size_of_80p:])
    y_test = np.array(y[size_of_80p:])

    print("X_train size: {}".format(X_train.shape))
    print("y_train size: {}".format(y_train.shape))
    print("X_test size: {}".format(X_test.shape))
    print("y_test size: {}".format(X_test.shape))
    ############################################################################

In [ ]:
 # LSTM weights

    generated_values1 = tf.truncated_normal([1,hidden_layers],stddev = 0.05)
    generated_values2 = tf.truncated_normal([hidden_layers,hidden_layers], stddev = 0.05)
    bias_values = tf.zeros([hidden_layers])

    # Weights
    W_inputG = tf.Variable(generated_values1)       #  weights input gate
    W_inputH = tf.Variable(generated_values2)       #  weights input hidden layer
    W_forgetG = tf.Variable(generated_values1)
    W_forgetH = tf.Variable(generated_values2)
    W_outputG = tf.Variable(generated_values1)
    W_outputH = tf.Variable(generated_values2)
    W_memoryG = tf.Variable(generated_values1)
    W_memoryH = tf.Variable(generated_values2)
    W_Output = tf.Variable(generated_values1)

    array_of_weights = np.array([W_inputG, W_inputH, W_forgetG,W_forgetH,W_outputG,W_outputH, W_memoryG, W_memoryH,W_Output])
     # Bias
    B_input = tf.Variable(bias_values)
    B_forget = tf.Variable(bias_values)
    B_output = tf.Variable(bias_values)
    B_memory = tf.Variable(bias_values)
    B_Output = tf.Variable(tf.zeros(1))

    array_of_bias = np.array([B_input, B_forget, B_output, B_memory, B_Output])

In [ ]:
# Placeholders
    inputs = tf.placeholder(tf.float32, [batch_size,window_size, 1])
    targets = tf.placeholder(tf.float32, [batch_size, 1])

    ##########################################################################

In [ ]:
################## Loop ######################################
    outputs_h =[]
    for ii in range(batch_size):
        batch_state = np.zeros([1, hidden_layers])
        batch_output = np.zeros([1, hidden_layers])

        for jj in range(window_size):
            batch_state, batch_output = block(tf.reshape(inputs[ii][jj], (-1,1)), batch_output, batch_state, array_of_weights,array_of_bias)

        outputs_h.append(tf.matmul(batch_output,W_Output) + B_Output)

    outputs_h

##################################################################################
###### Looking at the performance by defining the loss by mean squared error #####
    errors = []

    for ii in range(len(outputs_h)):
        aux_reshape = tf.reshape(targets[ii],(-1,1))
        errors.append(tf.error.mean_squared_error(aux_reshape), outputs_h[ii])

    error = tf.reduce_mean(errors)
##########################################################################

In [ ]:
############################# Training ###################################
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    trained_optimizer = 0; # change this
    for ii in range(epochs):
        trained_data = []
        jj = 0
        epoch_error = []

        while(jj + batch_size) <= len(X_train):
            X_batch = X_train[jj:jj + batch_size]
            y_batch = y_train[jj:jj + batch_size]
            a,b = session.run([outputs_h, error, trained_optimizer],feed_dict = {inputs:X_batch, targets:y_batch})
            trained_data.append(a)
            epoch_error.append(b)
            jj += batch_size


In [ ]:
################## Plotting the data ######################################
"""plt.figure(figsize=(12,7),frameon=False, facecolor='brown',edgecolor='blue')
plt.title('Data column 1')
plt.xlabel('Time steps t')
plt.ylabel('Position of joint 1')
plt.plot(scaled_data1,label='pos,joint1')
plt.legend()
plt.show()
"""

    plt.figure(figsize=(16,7))
    plt.title('Data')
    plt.xlabel('Time steps t')
    plt.ylabel('Position of joints')
    plt.plot(scaled_data1,label='Original data')
    plt.plot(tranied_data,label = 'Training data')
    plt.plot(test_data, label = 'Testing data')
    plt.legend()
    plt.show()


In [ ]:
# improve by reducing the window size

## To do:
# * implement in Jupyter, make it work
# *